In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
import numpy
import math

In [3]:
import pandas as pd
import numpy as np
import talib

df = pd.read_csv('AUD_USD_H4_for_keras.csv')
df.reset_index(inplace=True, drop=True)   

In [4]:
try:
    df = df.drop(['Unnamed: 0'], axis=1)
except:
    pass
try:
    df = df.drop(['volume'], axis=1)
except:
    pass

In [5]:
df

,datetime,close,open,high,low
0,2005-01-02T18:00:00.000000000Z,0.78060,0.78230,0.78280,0.78060
1,2005-01-02T22:00:00.000000000Z,0.78155,0.78070,0.78390,0.78070
2,2005-01-03T02:00:00.000000000Z,0.77535,0.78145,0.78165,0.77265
3,2005-01-03T06:00:00.000000000Z,0.77835,0.77535,0.77945,0.77315
4,2005-01-03T10:00:00.000000000Z,0.77775,0.77835,0.78135,0.77615
...,...,...,...,...,...
24472,2019-12-30T02:00:00.000000000Z,0.69934,0.69909,0.69992,0.69842
24473,2019-12-30T06:00:00.000000000Z,0.69962,0.69930,0.69970,0.69842
24474,2019-12-30T10:00:00.000000000Z,0.69986,0.69959,0.69996,0.69896
24475,2019-12-30T14:00:00.000000000Z,0.70003,0.69982,0.70046,0.69891


In [6]:
returns = df['close']
returns = returns.pct_change()

In [7]:
df = pd.concat([df,
                returns,
                ], axis=1)

df.columns = ['datetime','close','open', 'high','low','close_diff',]
df['close_diff'] = df['close_diff'].shift(-1)

df

,datetime,close,open,high,low,close_diff
0,2005-01-02T18:00:00.000000000Z,0.78060,0.78230,0.78280,0.78060,0.001217
1,2005-01-02T22:00:00.000000000Z,0.78155,0.78070,0.78390,0.78070,-0.007933
2,2005-01-03T02:00:00.000000000Z,0.77535,0.78145,0.78165,0.77265,0.003869
3,2005-01-03T06:00:00.000000000Z,0.77835,0.77535,0.77945,0.77315,-0.000771
4,2005-01-03T10:00:00.000000000Z,0.77775,0.77835,0.78135,0.77615,0.001286
...,...,...,...,...,...,...
24472,2019-12-30T02:00:00.000000000Z,0.69934,0.69909,0.69992,0.69842,0.000400
24473,2019-12-30T06:00:00.000000000Z,0.69962,0.69930,0.69970,0.69842,0.000343
24474,2019-12-30T10:00:00.000000000Z,0.69986,0.69959,0.69996,0.69896,0.000243
24475,2019-12-30T14:00:00.000000000Z,0.70003,0.69982,0.70046,0.69891,-0.000857


In [8]:
open_prices = df['open']
close_prices = df['close']
high_prices = df['high']
low_prices = df['low']
diff_prices = df['close_diff']

In [9]:
supported = ["ROCP", "MACD", "RSI", "UO", "BOLL", "MA", "STOCH", "AO", "ROC", "WILLR"]

In [10]:
def extract_by_type(feature_type, open_prices=None, close_prices=None, high_prices=None, low_prices=None):
    if feature_type == 'ROCP':
        rocp1 = talib.ROCP(close_prices, timeperiod=1)
        rocp2 = talib.ROCP(close_prices, timeperiod=2)
        feature.append(rocp1)
        feature.append(rocp2)
    if feature_type == 'OROCP':
        orocp = talib.ROCP(open_prices, timeperiod=1)
        feature.append(orocp)
    if feature_type == 'HROCP':
        hrocp = talib.ROCP(high_prices, timeperiod=1)
        feature.append(hrocp)
    if feature_type == 'LROCP':
        lrocp = talib.ROCP(low_prices, timeperiod=1)
        feature.append(lrocp)
    if feature_type == 'MACD':
        macd, signal, hist = talib.MACD(close_prices, fastperiod=12, slowperiod=26, signalperiod=9)
        norm_macd = numpy.nan_to_num(macd) / math.sqrt(numpy.var(numpy.nan_to_num(macd)))
        norm_signal = numpy.nan_to_num(signal) / math.sqrt(numpy.var(numpy.nan_to_num(signal)))
        norm_hist = numpy.nan_to_num(hist) / math.sqrt(numpy.var(numpy.nan_to_num(hist)))
        macdrocp = talib.ROCP(norm_macd + numpy.max(norm_macd) - numpy.min(norm_macd), timeperiod=1)
        signalrocp = talib.ROCP(norm_signal + numpy.max(norm_signal) - numpy.min(norm_signal), timeperiod=1)
        histrocp = talib.ROCP(norm_hist + numpy.max(norm_hist) - numpy.min(norm_hist), timeperiod=1)
        # feature.append(macd / 100.0)
        # feature.append(signal / 100.0)
        # feature.append(hist / 100.0)
        feature.append(norm_macd)
        feature.append(norm_signal)
        feature.append(norm_hist)

        feature.append(macdrocp)
        feature.append(signalrocp)
        feature.append(histrocp)
    if feature_type == 'RSI':
        rsi6 = talib.RSI(close_prices, timeperiod=6)
        rsi12 = talib.RSI(close_prices, timeperiod=12)
        rsi24 = talib.RSI(close_prices, timeperiod=24)
        rsi6rocp = talib.ROCP(rsi6 + 100., timeperiod=1)
        rsi12rocp = talib.ROCP(rsi12 + 100., timeperiod=1)
        rsi24rocp = talib.ROCP(rsi24 + 100., timeperiod=1)
        feature.append(rsi6 / 100.0 - 0.5)
        feature.append(rsi12 / 100.0 - 0.5)
        feature.append(rsi24 / 100.0 - 0.5)
        # feature.append(numpy.maximum(rsi6 / 100.0 - 0.8, 0))
        # feature.append(numpy.maximum(rsi12 / 100.0 - 0.8, 0))
        # feature.append(numpy.maximum(rsi24 / 100.0 - 0.8, 0))
        # feature.append(numpy.minimum(rsi6 / 100.0 - 0.2, 0))
        # feature.append(numpy.minimum(rsi6 / 100.0 - 0.2, 0))
        # feature.append(numpy.minimum(rsi6 / 100.0 - 0.2, 0))
        # feature.append(numpy.maximum(numpy.minimum(rsi6 / 100.0 - 0.5, 0.3), -0.3))
        # feature.append(numpy.maximum(numpy.minimum(rsi6 / 100.0 - 0.5, 0.3), -0.3))
        # feature.append(numpy.maximum(numpy.minimum(rsi6 / 100.0 - 0.5, 0.3), -0.3))
        feature.append(rsi6rocp)
        feature.append(rsi12rocp)
        feature.append(rsi24rocp)
    if feature_type == 'UO':
        ult_osc = talib.ULTOSC(high_prices, low_prices, close_prices, timeperiod1=7, timeperiod2=14, timeperiod3=28)
        feature.append(ult_osc / 100.0 - 0.5)
    if feature_type == 'BOLL':
        upperband, middleband, lowerband = talib.BBANDS(close_prices, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
        feature.append((upperband - close_prices) / close_prices)
        feature.append((middleband - close_prices) / close_prices)
        feature.append((lowerband - close_prices) / close_prices)         
    if feature_type == 'MA':
        ma5 = talib.MA(close_prices, timeperiod=5)
        ma10 = talib.MA(close_prices, timeperiod=10)
        ma20 = talib.MA(close_prices, timeperiod=20)
        ma25 = talib.MA(close_prices, timeperiod=25)
        ma30 = talib.MA(close_prices, timeperiod=30)
        ma40 = talib.MA(close_prices, timeperiod=40)
        ma50 = talib.MA(close_prices, timeperiod=50)
        ma60 = talib.MA(close_prices, timeperiod=60)
        #ma360 = talib.MA(close_prices, timeperiod=70)
        #ma720 = talib.MA(close_prices, timeperiod=720)
        ma5rocp = talib.ROCP(ma5, timeperiod=1)
        ma10rocp = talib.ROCP(ma10, timeperiod=1)
        ma20rocp = talib.ROCP(ma20, timeperiod=1)
        ma25rocp = talib.ROCP(ma25, timeperiod=1)
        ma30rocp = talib.ROCP(ma30, timeperiod=1)
        ma40rocp = talib.ROCP(ma40, timeperiod=1)
        ma50rocp = talib.ROCP(ma50, timeperiod=1)
        ma60rocp = talib.ROCP(ma60, timeperiod=1)
        #ma360rocp = talib.ROCP(ma360, timeperiod=1)
        #ma720rocp = talib.ROCP(ma720, timeperiod=1)
        feature.append(ma5rocp)
        feature.append(ma10rocp)
        feature.append(ma20rocp)
        feature.append(ma25rocp)
        feature.append(ma30rocp)
        feature.append(ma40rocp)
        feature.append(ma50rocp)
        feature.append(ma60rocp)
        #feature.append(ma360rocp)
        #feature.append(ma720rocp)
        feature.append((ma5 - close_prices) / close_prices)
        feature.append((ma10 - close_prices) / close_prices)
        feature.append((ma20 - close_prices) / close_prices)
        feature.append((ma25 - close_prices) / close_prices)
        feature.append((ma30 - close_prices) / close_prices)
        feature.append((ma40 - close_prices) / close_prices)
        feature.append((ma50 - close_prices) / close_prices)
        feature.append((ma60 - close_prices) / close_prices)
        #feature.append((ma360 - close_prices) / close_prices)
        #feature.append((ma720 - close_prices) / close_prices)
    if feature_type == 'STOCH':
        slow_stoch_k, slow_stoch_d = talib.STOCH(high_prices ,low_prices ,close_prices ,fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
        fast_stoch_k, fast_stoch_d = talib.STOCHF(high_prices , low_prices , close_prices , fastk_period=5, fastd_period=3, fastd_matype=0)
        fast_rsi_k, fast_rsi_d = talib.STOCHRSI(close_prices, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)                      
        feature.append(slow_stoch_k / 100.0 - 0.5)
        feature.append(slow_stoch_d / 100.0 - 0.5)
        feature.append(fast_stoch_k / 100.0 - 0.5)
        feature.append(fast_stoch_d / 100.0 - 0.5)
        feature.append(fast_rsi_k / 100.0 - 0.5)
        feature.append(fast_rsi_d / 100.0 - 0.5)
    if feature_type == 'AO':          
        median_price = (high_prices + low_prices) / 2
        ao = talib.SMA(median_price, 5)-talib.SMA(median_price, 34)          
        feature.append(ao)
    if feature_type == 'ROC':
        roc5 = talib.ROC(close_prices, timeperiod=5)
        roc10 = talib.ROC(close_prices, timeperiod=10)
        roc20 = talib.ROC(close_prices, timeperiod=20)
        roc25 = talib.ROC(close_prices, timeperiod=25)
        feature.append(roc5)
        feature.append(roc10)
        feature.append(roc20)
        feature.append(roc25)
    if feature_type == 'WILLR':
        willr = talib.WILLR(high_prices,low_prices,close_prices, timeperiod=14)
        feature.append(willr / 100.0 - 0.5)

    return feature

In [11]:
list_is = ["ROCP1", "ROCP2", "norm_macd", "norm_signal", "norm_hist", "macdrocp", "singalrocp", "histrocp",
            "rsi6", "rsi12", "rsi24", "rsi6rocp", "rsi12rocp", "rsi24rocp", "UO", "upperBOLL", "middleBOLL",
            "lowerBOLL", "MA5rocp", "MA10rocp", "MA20rocp", "MA25rocp", "MA30rocp", "MA40rocp", "MA50rocp", "MA60rocp", 
            "MA5", "MA10", "MA20", "MA25", "MA30", "MA40", "MA50", "MA60", "Slow_stochk", "Slow_stochd", 
            "Fast_stochk", "Fast_stochd", "Fast_stoch_rsik", "Fast_stoch_rsid", "AO","ROC5", "ROC10", "ROC20",
            "ROC25", "WILLR"]
len(list_is)

46

In [12]:
feature = []
for i in supported:
    extract_by_type(i, open_prices=open_prices, close_prices=close_prices, high_prices=high_prices, low_prices=low_prices)

In [13]:
feature

[0             NaN
 1        0.001217
 2       -0.007933
 3        0.003869
 4       -0.000771
            ...   
 24472    0.000315
 24473    0.000400
 24474    0.000343
 24475    0.000243
 24476   -0.000857
 Length: 24477, dtype: float64,
 0             NaN
 1             NaN
 2       -0.006726
 3       -0.004094
 4        0.003095
            ...   
 24472    0.001489
 24473    0.000715
 24474    0.000744
 24475    0.000586
 24476   -0.000614
 Length: 24477, dtype: float64,
 array([0.        , 0.        , 0.        , ..., 0.84094365, 0.8521146 ,
        0.83447462]),
 array([0.        , 0.        , 0.        , ..., 0.75307892, 0.7830562 ,
        0.80329949]),
 array([0.        , 0.        , 0.        , ..., 0.43919572, 0.38148515,
        0.25761228]),
 array([        nan,  0.        ,  0.        , ...,  0.00142222,
         0.00077551, -0.00122366]),
 array([       nan, 0.        , 0.        , ..., 0.002566  , 0.00222312,
        0.00149791]),
 array([        nan,  0.        ,  0.

In [14]:
df = pd.DataFrame(feature)
df

,0,1,2,3,4,5,6,7,8,9,...,24467,24468,24469,24470,24471,24472,24473,24474,24475,24476
0,NaN,0.001217,-0.007933,0.003869,-0.000771,0.001286,-0.000514,-0.002184,0.004249,-0.007885,...,0.000805,0.001897,0.000545,0.001090,0.001174,0.000315,0.000400,0.000343,0.000243,-0.000857
1,NaN,NaN,-0.006726,-0.004094,0.003095,0.000514,0.000771,-0.002697,0.002056,-0.003670,...,0.001353,0.002704,0.002443,0.001635,0.002265,0.001489,0.000715,0.000744,0.000586,-0.000614
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.551646,0.604427,0.649456,0.698464,0.751699,0.790957,0.820486,0.840944,0.852115,0.834475
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.512952,0.538461,0.568411,0.602758,0.641518,0.680846,0.718567,0.753079,0.783056,0.803299
4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.227836,0.324621,0.381143,0.437091,0.493250,0.500482,0.480026,0.439196,0.381485,0.257612
5,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.002027,0.003739,0.003178,0.003448,0.003733,0.002742,0.002057,0.001422,0.000776,-0.001224
6,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001354,0.001926,0.002257,0.002583,0.002907,0.002941,0.002812,0.002566,0.002223,0.001498
7,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.002420,0.005948,0.003453,0.003406,0.003407,0.000437,-0.001236,-0.002471,-0.003501,-0.007541
8,NaN,NaN,NaN,NaN,NaN,NaN,-0.092593,-0.151163,0.012195,-0.171888,...,0.438613,0.461773,0.466181,0.473511,0.479316,0.480679,0.482446,0.483955,0.485048,0.264537
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.335057,0.369636,0.377689,0.392215,0.405435,0.408712,0.412902,0.416487,0.419061,0.321561


In [15]:
df = df.transpose()
df

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.001217,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.007933,-0.006726,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.003869,-0.004094,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.000771,0.003095,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24472,0.000315,0.001489,0.790957,0.680846,0.500482,0.002742,0.002941,0.000437,0.480679,0.408712,...,0.357494,0.410823,0.5,0.500000,0.005432,0.502989,0.900303,1.024196,1.224525,-0.571960
24473,0.000400,0.000715,0.820486,0.718567,0.480026,0.002057,0.002812,-0.001236,0.482446,0.412902,...,0.412281,0.414505,0.5,0.500000,0.005536,0.352860,0.815609,1.162555,1.311979,-0.538860
24474,0.000343,0.000744,0.840944,0.753079,0.439196,0.001422,0.002566,-0.002471,0.483955,0.416487,...,0.462687,0.410820,0.5,0.500000,0.005600,0.332597,0.760172,1.182628,1.182628,-0.512887
24475,0.000243,0.000586,0.852115,0.783056,0.381485,0.000776,0.002223,-0.003501,0.485048,0.419061,...,0.353741,0.409570,0.5,0.500000,0.005626,0.247745,0.738236,1.217449,1.207206,-0.552058


In [16]:
df.columns = ["ROCP1", "ROCP2", "norm_macd", "norm_signal", "norm_hist", "macdrocp", "singalrocp", "histrocp",
            "rsi6", "rsi12", "rsi24", "rsi6rocp", "rsi12rocp", "rsi24rocp", "UO", "upperBOLL", "middleBOLL",
            "lowerBOLL", "MA5rocp", "MA10rocp", "MA20rocp", "MA25rocp", "MA30rocp", "MA40rocp", "MA50rocp", "MA60rocp", 
            "MA5", "MA10", "MA20", "MA25", "MA30", "MA40", "MA50", "MA60", "Slow_stochk", "Slow_stochd", 
            "Fast_stochk", "Fast_stochd", "Fast_stoch_rsik", "Fast_stoch_rsid", "AO","ROC5", "ROC10", "ROC20",
            "ROC25", "WILLR"]

df.insert(0,"close_diff",diff_prices)
df.insert(0,"low",low_prices)
df.insert(0,"high",high_prices)
df.insert(0,"open",open_prices)
df.insert(0,"close",close_prices)

In [17]:
df

,close,open,high,low,close_diff,ROCP1,ROCP2,norm_macd,norm_signal,norm_hist,...,Fast_stochk,Fast_stochd,Fast_stoch_rsik,Fast_stoch_rsid,AO,ROC5,ROC10,ROC20,ROC25,WILLR
0,0.78060,0.78230,0.78280,0.78060,0.001217,NaN,NaN,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.78155,0.78070,0.78390,0.78070,-0.007933,0.001217,NaN,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.77535,0.78145,0.78165,0.77265,0.003869,-0.007933,-0.006726,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.77835,0.77535,0.77945,0.77315,-0.000771,0.003869,-0.004094,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.77775,0.77835,0.78135,0.77615,0.001286,-0.000771,0.003095,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24472,0.69934,0.69909,0.69992,0.69842,0.000400,0.000315,0.001489,0.790957,0.680846,0.500482,...,0.357494,0.410823,0.5,0.500000,0.005432,0.502989,0.900303,1.024196,1.224525,-0.571960
24473,0.69962,0.69930,0.69970,0.69842,0.000343,0.000400,0.000715,0.820486,0.718567,0.480026,...,0.412281,0.414505,0.5,0.500000,0.005536,0.352860,0.815609,1.162555,1.311979,-0.538860
24474,0.69986,0.69959,0.69996,0.69896,0.000243,0.000343,0.000744,0.840944,0.753079,0.439196,...,0.462687,0.410820,0.5,0.500000,0.005600,0.332597,0.760172,1.182628,1.182628,-0.512887
24475,0.70003,0.69982,0.70046,0.69891,-0.000857,0.000243,0.000586,0.852115,0.783056,0.381485,...,0.353741,0.409570,0.5,0.500000,0.005626,0.247745,0.738236,1.217449,1.207206,-0.552058


In [18]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
cols_to_norm = ["ROCP1", "ROCP2", "norm_macd", "norm_signal", "norm_hist", "macdrocp", "singalrocp", "histrocp",
            "rsi6", "rsi12", "rsi24", "rsi6rocp", "rsi12rocp", "rsi24rocp", "UO", "upperBOLL", "middleBOLL",
            "lowerBOLL", "MA5rocp", "MA10rocp", "MA20rocp", "MA25rocp", "MA30rocp", "MA40rocp", "MA50rocp", "MA60rocp", 
            "MA5", "MA10", "MA20", "MA25", "MA30", "MA40", "MA50", "MA60", "Slow_stochk", "Slow_stochd", 
            "Fast_stochk", "Fast_stochd", "Fast_stoch_rsik", "Fast_stoch_rsid", "AO","ROC5", "ROC10", "ROC20",
            "ROC25", "WILLR"]
df[cols_to_norm] = StandardScaler().fit_transform(df[cols_to_norm])

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

df

,close,open,high,low,close_diff,ROCP1,ROCP2,norm_macd,norm_signal,norm_hist,...,Fast_stochk,Fast_stochd,Fast_stoch_rsik,Fast_stoch_rsid,AO,ROC5,ROC10,ROC20,ROC25,WILLR
0,0.78060,0.78230,0.78280,0.78060,0.001217,NaN,NaN,0.007538,0.008456,-0.001492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.78155,0.78070,0.78390,0.78070,-0.007933,0.386769,NaN,0.007538,0.008456,-0.001492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.77535,0.78145,0.78165,0.77265,0.003869,-2.522216,-1.522475,0.007538,0.008456,-0.001492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.77835,0.77535,0.77945,0.77315,-0.000771,1.229967,-0.926926,0.007538,0.008456,-0.001492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.77775,0.77835,0.78135,0.77615,0.001286,-0.245222,0.700443,0.007538,0.008456,-0.001492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24472,0.69934,0.69909,0.69992,0.69842,0.000400,0.099897,0.336924,0.798495,0.689302,0.498989,...,1.165338,1.587925,1.170332,1.479638,0.754189,0.722004,0.927604,0.747775,0.801689,1.354561
24473,0.69962,0.69930,0.69970,0.69842,0.000343,0.127142,0.161701,0.828024,0.727023,0.478534,...,1.348109,1.602446,1.170332,1.479638,0.768643,0.506402,0.840312,0.848885,0.859005,1.464230
24474,0.69986,0.69959,0.69996,0.69896,0.000243,0.108914,0.168123,0.848481,0.761535,0.437703,...,1.516265,1.587913,1.170332,1.479638,0.777373,0.477302,0.783173,0.863554,0.774230,1.550286
24475,0.70003,0.69982,0.70046,0.69891,-0.000857,0.077078,0.132468,0.859652,0.791513,0.379993,...,1.152820,1.582980,1.170332,1.479638,0.781025,0.355444,0.760565,0.889000,0.790338,1.420501


In [19]:
def var (row):
   if row['close_diff'] <= -0.0010 :
      return '0'
   if row['close_diff'] > +0.0010 :
      return '2'
   else:
       return '1'
   
df['NextDayPred'] = df.apply(lambda row: var(row), axis=1)
df.NextDayPred = df.NextDayPred.astype(int)

In [20]:
df.NextDayPred[:-2] = df.NextDayPred[:-2].astype(int)

C:\Users\kohli\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
type(df.NextDayPred[0])

numpy.int32

In [22]:
df['NextDayPred'].value_counts()

1    9591
2    7510
0    7376
Name: NextDayPred, dtype: int64

In [23]:
df[60:80]

,close,open,high,low,close_diff,ROCP1,ROCP2,norm_macd,norm_signal,norm_hist,...,Fast_stochd,Fast_stoch_rsik,Fast_stoch_rsid,AO,ROC5,ROC10,ROC20,ROC25,WILLR,NextDayPred
60,0.7588,0.7589,0.7612,0.7567,0.000000,-0.042040,-0.387293,-0.276632,-0.071929,-0.703771,...,-1.006218,-0.819943,-1.061440,-0.169621,-1.257297,-1.128806,-0.345759,-0.138753,-0.886308,1
61,0.7588,0.7586,0.7598,0.7576,0.000923,-0.000147,-0.030002,-0.341475,-0.129820,-0.738200,...,-0.951908,-0.442234,-0.946132,-0.283740,-0.678468,-1.075630,-0.336218,-0.061253,-1.189122,1
62,0.7595,0.7590,0.7601,0.7590,0.000132,0.293140,0.208627,-0.368838,-0.181932,-0.664659,...,-0.543119,0.908128,-0.149262,-0.284348,0.151083,-0.821529,-0.326375,0.172191,-0.947163,1
63,0.7596,0.7595,0.7603,0.7578,0.001843,0.041712,0.238457,-0.383208,-0.226667,-0.570781,...,-0.140513,1.170332,0.689528,-0.271986,-0.094814,-0.981412,-0.192596,0.007780,-0.919553,2
64,0.7610,0.7596,0.7613,0.7595,-0.002628,0.585809,0.446849,-0.351089,-0.255648,-0.370299,...,0.612351,1.170332,1.369134,-0.284065,0.397050,-0.753234,0.028131,-0.018069,-0.481390,0
65,0.7590,0.7611,0.7620,0.7587,0.001713,-0.835687,-0.178964,-0.377444,-0.284418,-0.367616,...,0.348997,-0.575412,0.743905,-0.260880,0.037507,-0.875493,-0.116045,-0.293126,-1.057605,2
66,0.7603,0.7590,0.7603,0.7584,-0.002631,0.544385,-0.208378,-0.357561,-0.303221,-0.240767,...,0.425506,0.691144,0.541954,-0.250058,0.283546,-0.284219,0.424690,-0.172799,-0.683065,0
67,0.7583,0.7603,0.7605,0.7574,-0.000527,-0.836456,-0.208926,-0.393427,-0.325864,-0.289646,...,-0.546122,-1.199813,-0.456929,-0.270040,-0.227250,-0.054661,-0.555399,-0.396023,-1.220140,1
68,0.7579,0.7583,0.7587,0.7576,-0.007785,-0.167850,-0.714666,-0.428080,-0.351322,-0.325473,...,-0.821504,-1.199813,-0.720080,-0.294644,-0.321751,-0.298638,-0.914761,-0.319249,-1.345168,0
69,0.7520,0.7578,0.7578,0.7517,0.001463,-2.475072,-1.880656,-0.614299,-0.411156,-0.762918,...,-1.565689,-1.199813,-1.517012,-0.430914,-1.698772,-0.937429,-1.420515,-0.750404,-1.649757,2


In [24]:
df = df.drop(['close_diff'], axis=1)

In [25]:
df = df.dropna()
df

,close,open,high,low,ROCP1,ROCP2,norm_macd,norm_signal,norm_hist,macdrocp,...,Fast_stochd,Fast_stoch_rsik,Fast_stoch_rsid,AO,ROC5,ROC10,ROC20,ROC25,WILLR,NextDayPred
60,0.75880,0.75890,0.76120,0.75670,-0.042040,-0.387293,-0.276632,-0.071929,-0.703771,-0.709775,...,-1.006218,-0.819943,-1.061440,-0.169621,-1.257297,-1.128806,-0.345759,-0.138753,-0.886308,1
61,0.75880,0.75860,0.75980,0.75760,-0.000147,-0.030002,-0.341475,-0.129820,-0.738200,-0.534314,...,-0.951908,-0.442234,-0.946132,-0.283740,-0.678468,-1.075630,-0.336218,-0.061253,-1.189122,1
62,0.75950,0.75900,0.76010,0.75900,0.293140,0.208627,-0.368838,-0.181932,-0.664659,-0.229382,...,-0.543119,0.908128,-0.149262,-0.284348,0.151083,-0.821529,-0.326375,0.172191,-0.947163,1
63,0.75960,0.75950,0.76030,0.75780,0.041712,0.238457,-0.383208,-0.226667,-0.570781,-0.123003,...,-0.140513,1.170332,0.689528,-0.271986,-0.094814,-0.981412,-0.192596,0.007780,-0.919553,2
64,0.76100,0.75960,0.76130,0.75950,0.585809,0.446849,-0.351089,-0.255648,-0.370299,0.259525,...,0.612351,1.170332,1.369134,-0.284065,0.397050,-0.753234,0.028131,-0.018069,-0.481390,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24472,0.69934,0.69909,0.69992,0.69842,0.099897,0.336924,0.798495,0.689302,0.498989,0.292509,...,1.587925,1.170332,1.479638,0.754189,0.722004,0.927604,0.747775,0.801689,1.354561,1
24473,0.69962,0.69930,0.69970,0.69842,0.127142,0.161701,0.828024,0.727023,0.478534,0.218202,...,1.602446,1.170332,1.479638,0.768643,0.506402,0.840312,0.848885,0.859005,1.464230,1
24474,0.69986,0.69959,0.69996,0.69896,0.108914,0.168123,0.848481,0.761535,0.437703,0.149362,...,1.587913,1.170332,1.479638,0.777373,0.477302,0.783173,0.863554,0.774230,1.550286,1
24475,0.70003,0.69982,0.70046,0.69891,0.077078,0.132468,0.859652,0.791513,0.379993,0.079238,...,1.582980,1.170332,1.479638,0.781025,0.355444,0.760565,0.889000,0.790338,1.420501,1


In [26]:
df

,close,open,high,low,ROCP1,ROCP2,norm_macd,norm_signal,norm_hist,macdrocp,...,Fast_stochd,Fast_stoch_rsik,Fast_stoch_rsid,AO,ROC5,ROC10,ROC20,ROC25,WILLR,NextDayPred
60,0.75880,0.75890,0.76120,0.75670,-0.042040,-0.387293,-0.276632,-0.071929,-0.703771,-0.709775,...,-1.006218,-0.819943,-1.061440,-0.169621,-1.257297,-1.128806,-0.345759,-0.138753,-0.886308,1
61,0.75880,0.75860,0.75980,0.75760,-0.000147,-0.030002,-0.341475,-0.129820,-0.738200,-0.534314,...,-0.951908,-0.442234,-0.946132,-0.283740,-0.678468,-1.075630,-0.336218,-0.061253,-1.189122,1
62,0.75950,0.75900,0.76010,0.75900,0.293140,0.208627,-0.368838,-0.181932,-0.664659,-0.229382,...,-0.543119,0.908128,-0.149262,-0.284348,0.151083,-0.821529,-0.326375,0.172191,-0.947163,1
63,0.75960,0.75950,0.76030,0.75780,0.041712,0.238457,-0.383208,-0.226667,-0.570781,-0.123003,...,-0.140513,1.170332,0.689528,-0.271986,-0.094814,-0.981412,-0.192596,0.007780,-0.919553,2
64,0.76100,0.75960,0.76130,0.75950,0.585809,0.446849,-0.351089,-0.255648,-0.370299,0.259525,...,0.612351,1.170332,1.369134,-0.284065,0.397050,-0.753234,0.028131,-0.018069,-0.481390,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24472,0.69934,0.69909,0.69992,0.69842,0.099897,0.336924,0.798495,0.689302,0.498989,0.292509,...,1.587925,1.170332,1.479638,0.754189,0.722004,0.927604,0.747775,0.801689,1.354561,1
24473,0.69962,0.69930,0.69970,0.69842,0.127142,0.161701,0.828024,0.727023,0.478534,0.218202,...,1.602446,1.170332,1.479638,0.768643,0.506402,0.840312,0.848885,0.859005,1.464230,1
24474,0.69986,0.69959,0.69996,0.69896,0.108914,0.168123,0.848481,0.761535,0.437703,0.149362,...,1.587913,1.170332,1.479638,0.777373,0.477302,0.783173,0.863554,0.774230,1.550286,1
24475,0.70003,0.69982,0.70046,0.69891,0.077078,0.132468,0.859652,0.791513,0.379993,0.079238,...,1.582980,1.170332,1.479638,0.781025,0.355444,0.760565,0.889000,0.790338,1.420501,1


In [27]:
X_next_day = df.iloc[-1,:]
X_next_day = np.array(X_next_day)
X_next_day = X_next_day.reshape((1,51))

In [28]:
X_next_day = np.delete(X_next_day, 50, 1)

In [29]:
df = df[:-1]

df_X = df.iloc[:, 0:50].values
df_y = df.iloc[:, -1].values

X = df.iloc[:, 0:50].values
y = df.iloc[:, -1].values
X_train, X_test, y_train, y_test = X[:20000],X[20001:],y[:20000],y[20001:]

In [30]:
X_train

array([[ 0.7588    ,  0.7589    ,  0.7612    , ..., -0.34575926,
        -0.13875267, -0.88630809],
       [ 0.7588    ,  0.7586    ,  0.7598    , ..., -0.33621799,
        -0.06125336, -1.18912216],
       [ 0.7595    ,  0.759     ,  0.7601    , ..., -0.32637511,
         0.1721905 , -0.9471634 ],
       ...,
       [ 0.76715   ,  0.76811   ,  0.76819   , ...,  0.14343604,
        -0.10405824, -1.41548594],
       [ 0.76843   ,  0.76818   ,  0.76872   , ...,  0.08690826,
        -0.03494691, -0.80464042],
       [ 0.76918   ,  0.76846   ,  0.7708    , ...,  0.15545668,
         0.15801823, -0.48524133]])

In [31]:
y_train

array([1, 1, 1, ..., 2, 1, 0])

In [42]:
from keras.utils import to_categorical

df_y = to_categorical(df_y)
y_train = to_categorical(y_train)
print(y_train)

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


# Keras Neural Network

In [33]:
#### keras
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import layers
from keras.optimizers import SGD
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
import numpy
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

In [34]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()

In [58]:
#Initializing ANN
classifier = Sequential()
# input layer and first hidden layer
classifier.add(Dense(62,activation='tanh',kernel_initializer='uniform',input_dim=50))
classifier.add(Dropout(0.1))

classifier.add(Dense(62,activation='relu',kernel_initializer='uniform'))
classifier.add(Dropout(0.1))

classifier.add(Dense(62,activation='relu',kernel_initializer='uniform'))
classifier.add(Dropout(0.1))

classifier.add(Dense(62,activation='relu',kernel_initializer='uniform'))
classifier.add(Dropout(0.1))

classifier.add(Dense(3,activation="softmax", kernel_initializer="uniform"))

classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [40]:
classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [59]:
classifier.fit(X_train, y_train, batch_size=30, epochs=50)

Epoch 1/50
20000/20000 [==============================] - 3s 129us/step - loss: 0.6330 - accuracy: 0.6668
Epoch 2/50
20000/20000 [==============================] - 2s 111us/step - loss: 0.6308 - accuracy: 0.6676
Epoch 3/50
20000/20000 [==============================] - 2s 107us/step - loss: 0.6300 - accuracy: 0.6682
Epoch 4/50
20000/20000 [==============================] - 3s 135us/step - loss: 0.6290 - accuracy: 0.6679
Epoch 5/50
20000/20000 [==============================] - 2s 109us/step - loss: 0.6286 - accuracy: 0.6684
Epoch 6/50
20000/20000 [==============================] - 2s 109us/step - loss: 0.6280 - accuracy: 0.6685
Epoch 7/50
20000/20000 [==============================] - 2s 109us/step - loss: 0.6276 - accuracy: 0.6691
Epoch 8/50
20000/20000 [==============================] - 2s 111us/step - loss: 0.6277 - accuracy: 0.6686
Epoch 9/50
20000/20000 [==============================] - 2s 118us/step - loss: 0.6272 - accuracy: 0.6688
Epoch 10/50
20000/20000 [=====================

In [36]:
#Initializing ANN
def create_model(dropout_rate=0.1):
    
    classifier = Sequential()
    # input layer and first hidden layer    
    classifier.add(Dense(62,activation='tanh',kernel_initializer='uniform',input_dim=50))
    classifier.add(Dropout(0.1))

    classifier.add(Dense(62,activation='relu',kernel_initializer='uniform'))
    classifier.add(Dropout(0.1))

    classifier.add(Dense(62,activation='relu',kernel_initializer='uniform'))
    classifier.add(Dropout(0.1))

    classifier.add(Dense(62,activation='relu',kernel_initializer='uniform'))
    classifier.add(Dropout(0.1))

    classifier.add(Dense(3,activation="softmax", kernel_initializer="uniform"))

#     optimizer = SGD(lr=learn_rate, momentum=momentum)
    classifier.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    
#     classifier.fit(X_train, y_train, batch_size=30, epochs=50)
    return classifier

In [185]:
classifier.save('keras_nn_aud_usd_h4_train_without_fit_transform_4.h5')

In [60]:
y_pred = classifier.predict(X_test)
y_pred

array([[0.2766783 , 0.4181276 , 0.30519408],
       [0.2766783 , 0.4181276 , 0.30519408],
       [0.27759895, 0.4162576 , 0.30614343],
       ...,
       [0.27243075, 0.43486115, 0.29270816],
       [0.26821652, 0.44260007, 0.28918347],
       [0.26314023, 0.45496187, 0.28189784]], dtype=float32)

In [61]:
y_pred = pd.DataFrame(y_pred)
y_pred.columns = ['Short', 'Stay', 'Buy']
y_pred = y_pred.astype(float)
y_pred

,Short,Stay,Buy
0,0.276678,0.418128,0.305194
1,0.276678,0.418128,0.305194
2,0.277599,0.416258,0.306143
3,0.276678,0.418128,0.305194
4,0.276678,0.418128,0.305194
...,...,...,...
4410,0.288827,0.401182,0.309991
4411,0.276481,0.427937,0.295582
4412,0.272431,0.434861,0.292708
4413,0.268217,0.442600,0.289183


In [62]:
def enc (row_test):
   if (row_test['Short'] > row_test['Stay'] and 
       row_test['Short'] > row_test['Buy']  ):
      return '0'
   else:
       if (row_test['Stay'] > row_test['Short'] and 
           row_test['Stay'] > row_test['Buy']    ):
          return '1'
       else:
           if (row_test['Buy'] > row_test['Short'] and 
               row_test['Buy'] > row_test['Stay'] ):
               return '2'
           else:
               return 'nan'
           

y_pred['Predictions'] = y_pred.apply(lambda row_test: enc(row_test), axis=1)
predictions = y_pred['Predictions']
predictions = predictions.astype(int)
y_pred

,Short,Stay,Buy,Predictions
0,0.276678,0.418128,0.305194,1
1,0.276678,0.418128,0.305194,1
2,0.277599,0.416258,0.306143,1
3,0.276678,0.418128,0.305194,1
4,0.276678,0.418128,0.305194,1
...,...,...,...,...
4410,0.288827,0.401182,0.309991,1
4411,0.276481,0.427937,0.295582,1
4412,0.272431,0.434861,0.292708,1
4413,0.268217,0.442600,0.289183,1


In [63]:
pred = np.array(predictions)

In [64]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.4776896942242356

In [56]:
pd.Series(pred).value_counts()

1    4310
2     105
dtype: int64

In [57]:
min(df['low'])

0.60093

In [27]:
classifier.fit(df_X, df_y, batch_size=50, epochs=100)

Epoch 1/100
24455/24455 [==============================] - 4s 162us/step - loss: 0.2517 - accuracy: 0.8756
Epoch 2/100
24455/24455 [==============================] - 4s 157us/step - loss: 0.2514 - accuracy: 0.8742
Epoch 3/100
24455/24455 [==============================] - 4s 157us/step - loss: 0.2538 - accuracy: 0.8731
Epoch 4/100
24455/24455 [==============================] - 4s 164us/step - loss: 0.2534 - accuracy: 0.8719
Epoch 5/100
24455/24455 [==============================] - 4s 165us/step - loss: 0.2539 - accuracy: 0.8721
Epoch 6/100
24455/24455 [==============================] - 4s 166us/step - loss: 0.2484 - accuracy: 0.8757
Epoch 7/100
24455/24455 [==============================] - 4s 163us/step - loss: 0.2578 - accuracy: 0.8710
Epoch 8/100
24455/24455 [==============================] - 4s 164us/step - loss: 0.2510 - accuracy: 0.8736
Epoch 9/100
24455/24455 [==============================] - 4s 157us/step - loss: 0.2439 - accuracy: 0.8773
Epoch 10/100
24455/24455 [===========

Epoch 77/100
24455/24455 [==============================] - 4s 162us/step - loss: 0.2353 - accuracy: 0.8839
Epoch 78/100
24455/24455 [==============================] - 4s 149us/step - loss: 0.2393 - accuracy: 0.8819
Epoch 79/100
24455/24455 [==============================] - 4s 151us/step - loss: 0.2379 - accuracy: 0.8830
Epoch 80/100
24455/24455 [==============================] - 4s 150us/step - loss: 0.2347 - accuracy: 0.8836
Epoch 81/100
24455/24455 [==============================] - 4s 155us/step - loss: 0.2346 - accuracy: 0.8839
Epoch 82/100
24455/24455 [==============================] - 4s 158us/step - loss: 0.2330 - accuracy: 0.8841
Epoch 83/100
24455/24455 [==============================] - 4s 144us/step - loss: 0.2348 - accuracy: 0.8833
Epoch 84/100
24455/24455 [==============================] - 4s 150us/step - loss: 0.2310 - accuracy: 0.8842
Epoch 85/100
24455/24455 [==============================] - 4s 159us/step - loss: 0.2348 - accuracy: 0.8819
Epoch 86/100
24455/24455 [==

In [75]:
X_next_day = df.iloc[-1,:]
X_next_day = np.array(X_next_day)
X_next_day = X_next_day.reshape((1,54))

In [76]:
y_next_day = classifier.predict(X_next_day)

In [74]:
y_next_day = pd.DataFrame(y_next_day)
y_next_day.columns = ['Short', 'Stay', 'Buy']
print(y_next_day)

          Short  Stay           Buy
0  4.200701e-18   1.0  2.821858e-11


In [33]:
classifier.save('keras_nn_aud_usd_h4.h5')

In [ ]:
# It can be used to reconstruct the model identically.
model = keras.models.load_model('keras_nn_aud_usd_h4.h5')